In [1]:
import imageio

robot_img = imageio.imread('/robot.jpeg')
robot_img.shape

/var/folders/pn/6632v8q9641cv_mxbfsbzvc80000gn/T/ipykernel_25342/1818918429.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  robot_img = imageio.imread('/Users/san/Desktop/robot.jpeg')


(600, 591, 3)

In [2]:
import torch

robot_img_tensor = torch.from_numpy(robot_img)
print(robot_img_tensor.shape)
robot_img_tensor = robot_img_tensor.permute(2,0,1)
print(robot_img_tensor.shape)

torch.Size([600, 591, 3])
torch.Size([3, 600, 591])


## 테이블 데이터 표현하기

In [3]:
import csv
import numpy as np

data_path = '/winequality-white.csv'
wine_numpy = np.loadtxt(data_path, dtype=np.float32, delimiter = ';', skiprows = 1)
wine_numpy.shape

(4898, 12)

12개의 feature로 이뤄진 4898개의 와인 데이터 샘플

In [4]:
col_list = next(csv.reader(open(data_path), delimiter= ';'))
col_list

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [5]:
# numpy to tensor

wine_tensor = torch.from_numpy(wine_numpy)

In [7]:
#label 제외한 data
data = wine_tensor[:, :-1]
target = wine_tensor[:, -1].long()
target_onehot = torch.zeros(target.shape[0], 10)
target_onehot.scatter_(1, target.unsqueeze(1), 1.0)

target_onehot



tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [10]:
bad_data = data[target <= 3]
bad_mean = torch.mean(bad_data, dim = 0)

mid_data = data[(target >3) & (target < 7)]
mid_mean = torch.mean(bad_data, dim = 0)

good_data = data[target >= 7]
good_mean = torch.mean(bad_data, dim = 0)


In [11]:
for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):
    print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args))

 0 fixed acidity          7.60   7.60   7.60
 1 volatile acidity       0.33   0.33   0.33
 2 citric acid            0.34   0.34   0.34
 3 residual sugar         6.39   6.39   6.39
 4 chlorides              0.05   0.05   0.05
 5 free sulfur dioxide   53.33  53.33  53.33
 6 total sulfur dioxide 170.60 170.60 170.60
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.19   3.19
 9 sulphates              0.47   0.47   0.47
10 alcohol               10.34  10.34  10.34


## 텍스트 데이터 표현하기

In [12]:
with open('/1342.txt', encoding ='utf8') as f:
    text = f.read()

lines = text.split('\n')
line = lines[200]
line

'“Impossible, Mr. Bennet, impossible, when I am not acquainted with him'

In [13]:
# 단어 원핫인코딩

#불용어제거
def clean_words(input_str):
    punctuation = '.,;:"!?”“_-'
    word_list = input_str.lower().replace('\n',' ').split()
    word_list = [word.strip(punctuation) for word in word_list]
    return word_list

words_in_line = clean_words(line)
line, words_in_line



('“Impossible, Mr. Bennet, impossible, when I am not acquainted with him',
 ['impossible',
  'mr',
  'bennet',
  'impossible',
  'when',
  'i',
  'am',
  'not',
  'acquainted',
  'with',
  'him'])

In [14]:
# 단어 인덱스로 매핑하기

word_list = sorted(set(clean_words(text)))

#단어사전과 같은 역할-> 나무:2, 바다:3 같은 형식
word2index_dict = {word: i for (i, word) in enumerate(word_list)}

len(word2index_dict), word2index_dict['impossible']

(7261, 3394)

In [15]:
# line에 원핫인코딩 적용

word_tensor = torch.zeros(len(words_in_line), len(word2index_dict))
for i, word in enumerate(words_in_line):
    word_index = word2index_dict[word]
    word_tensor[i][word_index] = 1
    print('{:2} {:4} {}'.format(i, word_index, word))
    

 0 3394 impossible
 1 4305 mr
 2  813 bennet
 3 3394 impossible
 4 7078 when
 5 3315 i
 6  415 am
 7 4436 not
 8  239 acquainted
 9 7148 with
10 3215 him
